# Скрипт для формування pop up 


In [ ]:
# -*- coding: utf-8 -*-

import json
import arcpy

aprx = arcpy.mp.ArcGISProject("current")
map = aprx.listMaps()[0]
print(map.name)
layers = map.listLayers()

In [23]:
p = arcpy.Describe(layers[1]).catalogPath
f = arcpy.ListFields(path)
print(f[3].type)

Double


In [44]:
count = 0
for l in layers:
    if l.isGroupLayer:
        count += 1
print(count)

26


In [45]:
setFilelds = set()

for layer in layers:
    if layer.isFeatureLayer:
        path = arcpy.Describe(layer).catalogPath
        fields = arcpy.ListFields(path)
        for f in fields:
            setFilelds.add(f.name)

for j in setFilelds:
    print(j)

tp
KUDA
Стан
Шар
Назв_5
Гриф_
рёр_с
S
data
Shape
LENGTH
Z
внс
Ознак
expl
ATimeStamp
napr
Хара_1
Напру
Поясн
adress
Тимча
Позн_1
рџр_р
Підпи
Номер
namber
Ванта
Висот
П_Мат
експл
lep10
поясн
тиск
s
Власн
ТИП
Назв_6
Назв_2
Id
Назв_3
НАЗВА
ID
Відно
Абсол
HTML_пі
Назви
Площа
nazva
Поход
Назв_4
Вид_р
Дата_
Товщи
Назв_1
name
NEWFIELD1
рџрѕс
Кільк
ЕКСП_1
Відст
NAZV
Назва
SHAPE_Leng
OBJECTID
HTML_пі_
FID
EKCПЛ
Харак
Позна
Ширин
status
Довжи
Матер
Вид_р_
Btimestamp
typ
Шар_ID
ATIMESTAMP
Глиби


In [40]:
print(len(setFilelds))

80


In [58]:
new_field_value = {"tp":{"alias":"Трасформатор", "bool":True},"KUDA":{"alias":"KUDA", "bool":False},"Стан":{"alias":"Стан", "bool":True},"Шар":{"alias":"Шар", "bool":False},"Назв_5":{"alias":"Назв_5", "bool":False},"Гриф_":{"alias":"Гриф_", "bool":False},"рёр_с":{"alias":"рёр_с", "bool":False},"S":{"alias":"Площа", "bool":True},"data":{"alias":"Дата", "bool":True},"Shape":{"alias":"Shape", "bool":False},"LENGTH":{"alias":"LENGTH", "bool":False},"Z":{"alias":"Висота над рівнем моря", "bool":True},"внс":{"alias":"внс", "bool":False},"Ознак":{"alias":"Ознак", "bool":False},"expl":{"alias":"Експлікація", "bool":True},"ATimeStamp":{"alias":"ATimeStamp", "bool":False},"napr":{"alias":"Напруга", "bool":True},"Хара_1":{"alias":"Характеристика", "bool":True},"Напру":{"alias":"Напруга", "bool":True},"Поясн":{"alias":"Поясн", "bool":True},"adress":{"alias":"Адреса", "bool":True},"Тимча":{"alias":"Тимча", "bool":False},"Позн_1":{"alias":"Позн_1", "bool":False},"рџр_р":{"alias":"рџр_р", "bool":False},"Підпи":{"alias":"Підпи", "bool":False},"Номер":{"alias":"Номер", "bool":True},"namber":{"alias":"Номер", "bool":True},"Ванта":{"alias":"Вантажопідйомність", "bool":True},"Висот":{"alias":"Висота", "bool":True},"П_Мат":{"alias":"Покриття", "bool":True},"експл":{"alias":"Експлікація", "bool":True},"lep10":{"alias":"Напруга", "bool":True},"поясн":{"alias":"Пояснювальний напис", "bool":True},"тиск":{"alias":"Тиск", "bool":True},"s":{"alias":"Площа", "bool":True},"Власн":{"alias":"Власна назва", "bool":True},"ТИП":{"alias":"Тип", "bool":True},"Назв_6":{"alias":"Назв_6", "bool":False},"Назв_2":{"alias":"Назв_2", "bool":False},"Id":{"alias":"Id", "bool":False},"Назв_3":{"alias":"Назв_3", "bool":False},"НАЗВА":{"alias":"Назва", "bool":True},"ID":{"alias":"ID", "bool":False},"Відно":{"alias":"Відносна висота, м", "bool":True},"Абсол":{"alias":"Абсолютна висота, м", "bool":True},"HTML_пі":{"alias":"HTML_пі", "bool":False},"Назви":{"alias":"Назви", "bool":False},"Площа":{"alias":"Площа", "bool":True},"nazva":{"alias":"nazva", "bool":False},"Поход":{"alias":"Поход", "bool":False},"Назв_4":{"alias":"Назв_4", "bool":False},"Вид_р":{"alias":"Вид рослинності", "bool":True},"Дата_":{"alias":"Дата", "bool":True},"Товщи":{"alias":"Товщи", "bool":False},"Назв_1":{"alias":"Назва", "bool":True},"name":{"alias":"Назва", "bool":True},"NEWFIELD1":{"alias":"NEWFIELD1", "bool":False},"рџрѕс":{"alias":"рџрѕс", "bool":False},"Кільк":{"alias":"Кількість", "bool":True},"ЕКСП_1":{"alias":"Експлікація", "bool":True},"Відст":{"alias":"Відст", "bool":False},"NAZV":{"alias":"NAZV", "bool":False},"Назва":{"alias":"Назва", "bool":True},"SHAPE_Leng":{"alias":"SHAPE_Leng", "bool":False},"OBJECTID":{"alias":"OBJECTID", "bool":False},"HTML_пі_":{"alias":"HTML_пі_", "bool":False},"FID":{"alias":"FID", "bool":False},"EKCПЛ":{"alias":"Експлікація", "bool":True},"Харак":{"alias":"Характеристика", "bool":True},"Позна":{"alias":"Позна", "bool":False},"Ширин":{"alias":"Ширина", "bool":True},"status":{"alias":"status", "bool":False},"Довжи":{"alias":"Довжина", "bool":True},"Матер":{"alias":"Матер", "bool":False},"Вид_р_":{"alias":"Вид_р_", "bool":False},"Btimestamp":{"alias":"Btimestamp", "bool":False},"typ":{"alias":"Тип", "bool":True},"Шар_ID":{"alias":"Шар_ID", "bool":False},"ATIMESTAMP":{"alias":"ATIMESTAMP", "bool":False},"Глиби":{"alias":"Глибина", "bool":True}}

In [59]:
list_layers = []

for n, layer in enumerate(layers):
    if layer.isGroupLayer:
        list_layers.append(
            {
                "id": n
            }
        )
    elif layer.isFeatureLayer:
        path = arcpy.Describe(layer).catalogPath
        fields = arcpy.ListFields(path)
        
        fieldsList = []
        
        for field in fields:
            if field.type == "Double":
                fieldsList.append(
                    {
                        "fieldName": field.name,
                        "label": new_field_value[field.name]['alias'],
                        "tooltip": "",
                        "visible": new_field_value[field.name]['bool'],
                        "stringFieldOption": "textbox",
                        "format": {
                            "places": 2,
                            "digitSeparator": True
                        }
                    }
                )
            else:
                fieldsList.append(
                    {
                        "fieldName": field.name,
                        "label": new_field_value[field.name]['alias'],
                        "tooltip": "",
                        "visible": new_field_value[field.name]['bool'],
                        "stringFieldOption": "textbox"
                    }
                )
            
        
        list_layers.append(
            {
               "id": n,
                "popupInfo": {
                    "title": layer.name,
                    "fieldInfos": fieldsList,
                    "description": None,
                    "showAttachments": True,
                    "mediaInfos": []
                } 
            }
        )
        

In [60]:
jsonList = json.dumps(list_layers, ensure_ascii=False)

In [61]:
with open('C:\\Users\\Home PC\\Desktop\\json.json', 'w') as outfile:
    outfile.write(jsonList)